In [0]:
!pip install dfply
!pip install textdistance
import pandas as pd
import numpy as np
from datetime import datetime
import datetime as dt
from dateutil import parser
import textdistance
import re
import math
from dfply import *
from dateutil.relativedelta import *
from difflib import SequenceMatcher

In [0]:
def find_sofifa_id(row):
  date_str = str(row.DOB)
  dob = date_str.replace("00:00:00","").strip()
  name = row.Name.split(" ")[1].lower() if (len(row.Name.split(" ")) > 1) else row.Name.lower()
  nationality = row.Nationality.split(" ")[0].lower()
  try:
    sofifa_dob = sofifa[dob]
    name_cond = (sofifa_dob.short_name.apply(lambda x: string_dist(x.lower(),name)) > 0.5)   
    nat_cond = (sofifa_dob.nationality.apply(lambda x: string_dist(x.lower(),nationality)) > 0.5)
    filter_cond = name_cond & nat_cond
    matches = sofifa_dob[filter_cond]
    ids = matches.sofifa_id.unique()
  except Exception as e:
    name_cond = (sofifa.short_name.apply(lambda x: string_dist(x,name)) > 0.6)   
    nat_cond = (sofifa.nationality.apply(lambda x: string_dist(x,nationality)) > 0.6)
    filter_cond = name_cond & nat_cond
    matches = sofifa[filter_cond]
    ids = matches.sofifa_id.unique()
  ids = list(map(int, ids))
  return ids


def find_sofifa_id_v2(row):
  date_str = str(row.DOB)
  dob = date_str.replace("00:00:00","").strip()
  name = row.Name.lower()
  nationality = row.Nationality.split(" ")[0].lower()
  try:
    sofifa_dob = sofifa[dob]
    name_cond = (sofifa_dob.short_name.apply(lambda x: string_dist_v2(x.lower(),name)) > 0.5)   
    nat_cond = (sofifa_dob.nationality.apply(lambda x: string_dist_v2(x.lower(),nationality)) > 0.5)
    filter_cond = name_cond & nat_cond
    print(name)
    print(nationality)
    matches = sofifa_dob[filter_cond]
    print(matches)
    ids = matches.sofifa_id.unique()
  except Exception as e:
    name_cond = (sofifa.short_name.apply(lambda x: string_dist_v2(x,name)) > 0.6)   
    nat_cond = (sofifa.nationality.apply(lambda x: string_dist_v2(x,nationality)) > 0.6)
    filter_cond = name_cond & nat_cond
    matches = sofifa[filter_cond]
    ids = matches.sofifa_id.unique()
  ids = list(map(int, ids))
  return ids

def find_sofifa_id_v3(row):
  date_str = str(row.DOB)
  dob = date_str.replace("00:00:00","").strip()
  name = row.Name.lower()
  try:
    sofifa_dob = sofifa[dob]
    name_cond = (sofifa_dob.short_name.apply(lambda x: string_dist_v2(x.lower(),name)) > 0.5)   
    matches = sofifa_dob[name_cond]
    ids = matches.sofifa_id.unique()
  except Exception as e:
    name_cond = (sofifa.short_name.apply(lambda x: string_dist_v2(x,name)) > 0.6)   
    filter_cond = name_cond
    matches = sofifa[filter_cond]
    print(name)
    print(matches)
    ids = matches.sofifa_id.unique()
  ids = list(map(int, ids))
  return ids

def find_sofifa_id_array(row):
  date_str = str(row.DOB)
  dob = date_str.replace("00:00:00","").strip()
  name = row.Name.split(" ")[1].lower() if (len(row.Name.split(" ")) > 1) else row.Name.lower()
  nationality = row.Nationality.split(" ")[0].lower()
  sofifa_ids = row.sofifa_id
  try:
    print(name)
    sofifa_dob = sofifa[sofifa.sofifa_id.isin(sofifa_ids)]
    un_names = sofifa_dob.short_name.unique()
    un_nat = sofifa_dob.nationality.unique()
    max_name = max(list(map(lambda x: string_dist(x.lower(), name), un_names)))
    max_nat = max(list(map(lambda x: string_dist(x.lower(), nationality), un_nat)))
    name_cond = (sofifa_dob.short_name.apply(lambda x: string_dist(x.lower(),name)) == max_name)   
    nat_cond = (sofifa_dob.nationality.apply(lambda x: string_dist(x.lower(),nationality)) == max_nat)
    filter_cond = name_cond & nat_cond
    matches = sofifa_dob[filter_cond]
    print(matches)
    ids = matches.sofifa_id.unique()
  except Exception as e:
    name_cond = (sofifa.short_name.apply(lambda x: string_dist(x,name)) > 0.6)   
    nat_cond = (sofifa.nationality.apply(lambda x: string_dist(x,nationality)) > 0.6)
    filter_cond = name_cond & nat_cond
    matches = sofifa[filter_cond]
    ids = matches.sofifa_id.unique()
  ids = list(map(int, ids))
  return ids


def string_dist_v2(text1, text2):
  result = textdistance.ratcliff_obershelp(text1,text2)
  return result


def string_dist(text1, text2):
  result = textdistance.jaro_winkler(text1,text2)
  return result

def datetime_to_date(value):
    time = dt.datetime.fromtimestamp(value)
    year = time.strftime('%Y')
    month = time.strftime('%m')
    day = time.strftime('%d') 
    hour = time.strftime('%H') 
    date = str(year)+'-'+str(month)+'-'+str(day)+' '+str(hour)
    t = dt.datetime.strptime(date,'%Y-%m-%d %H')
    new_time = t.timestamp()
    return new_time

def tm_initial(name):
  result = ""
  split = name.split(" ")
  if(len(split) > 1):
    for part in split:
      result += part[0]
  else:
    result = split[0]
  return result.lower()

def sofifa_initial(name):
  result = ""
  if("." in name):
    split = name.split(".")
    for part in split:
      result += part[0]
  else:
    split = name.split(" ")
    for part in split:
      result += part[0]
  return result.lower()

pc


In [0]:
sofifa = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/sofifa_head_Clean.pkl")
players = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/Players_Clean.pkl")

In [0]:
sofifa = sofifa[sofifa.game_year == "2020-01-01"]
sofifa.set_index("dob", drop=False, inplace=True)
players.DOB = pd.to_datetime(players.DOB,format='%YYYY-%MM-%DD', errors = 'coerce')
sofifa.dob = pd.to_datetime(sofifa.dob,format='%YYYY-%MM-%DD', errors = 'coerce')
players["sofifa_id"] = players.apply(find_sofifa_id, axis = 1)
# players

In [0]:
def apply_len(value):
  return len(value)
not_found_v1 = players[players.sofifa_id.apply(apply_len) < 1]
found_v1 = players[players.sofifa_id.apply(apply_len) == 1]
needs_clear_v1 = players[players.sofifa_id.apply(apply_len) > 1]

In [0]:
not_found_v1['sofifa_id'] = not_found_v1.apply(find_sofifa_id_v2, axis = 1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
# not_found_v1[not_found_v1.sofifa_id.apply(apply_len) > 1]["sofifa_id"] = 244076
found_v2 = not_found_v1[not_found_v1.sofifa_id.apply(apply_len) == 1]
not_found_v2 = not_found_v1[not_found_v1.sofifa_id.apply(apply_len) < 1]
needs_clear_v2 = not_found_v1[not_found_v1.sofifa_id.apply(apply_len) > 1]

In [0]:
not_found_v2["sofifa_id"] = not_found_v2.apply(find_sofifa_id_v2, axis = 1)

In [0]:
found_v3 = not_found_v2[not_found_v2.sofifa_id.apply(apply_len) == 1]
not_found_v3 = not_found_v2[not_found_v2.sofifa_id.apply(apply_len) < 1]

In [0]:
not_found_v3

In [0]:
# needs_clear_v1.Contract_Expires

DOB
1988-09-03   2021-06-30
1992-06-24   2021-06-30
1991-04-11   2021-06-30
1989-09-13   2021-06-30
1995-09-20   2020-06-30
                ...    
1995-01-07          NaT
1997-02-24          NaT
1995-07-19   2023-06-30
1994-04-19   2023-06-30
1995-10-17   2021-06-30
Name: Contract_Expires, Length: 3584, dtype: datetime64[ns]

In [0]:
# needs_clear_v1["sofifa_id"] = needs_clear_v1.apply(find_sofifa_id_array, axis = 1)
found_v3 = needs_clear_v1[needs_clear_v1.sofifa_id.apply(apply_len) == 1]

In [0]:
def take_first(arr):
  return arr[0]

In [0]:
found = pd.concat([found_v1,found_v2,found_v3])
founded = found[["tm_Id","sofifa_id"]]
founded["sofifa_id"] = founded["sofifa_id"].apply(take_first) 
# founded.set_index("tm_Id", inplace=True, drop=True)
founded.drop_duplicates(subset = "sofifa_id",inplace=True, keep = False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [0]:
def attach_sofifa(tm_Id):
  result = math.nan
  filter = founded[founded.tm_Id == tm_Id]
  if(len(filter) != 0):
    result = filter.sofifa_id[0]
  return result

def make_int(id):
  result = id
  try:
    result = int(id)
    print(result)
  except Exception as e:
    print(str(e))
    pass
  return result

In [0]:
players = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/Players_Clean.pkl") 
players.drop(["sofifa_id"], axis = 1, inplace = True)

In [0]:
players["sofifa_id"] = players.tm_Id.apply(attach_sofifa)

In [0]:
players.sofifa_id = players.sofifa_id.apply(make_int)
players

In [0]:
players[players.sofifa_id.notnull()]

,tm_Id,Name,Team,Nationality,DOB,Height,SF,Position,Joined,Contract_Expires,Followers,sofifa_id
DOB,,,,,,,,,,,,
1986-03-27,17259,Manuel Neuer,Bayern Munich,Germany,1986-03-27,193.0,right,Goalkeeper,2011-07-01,2021-06-30,10043887.0,167495.0
1988-08-03,40680,Sven Ulreich,Bayern Munich,Germany,1988-08-03,192.0,right,Goalkeeper,2015-07-01,2021-06-30,284045.0,186569.0
1996-02-14,281963,Lucas Hernández,Bayern Munich,France Spain,1996-02-14,182.0,left,Defender - Centre-Back,2019-07-01,2024-06-30,1467772.0,220814.0
1995-09-03,166601,Niklas Süle,Bayern Munich,Germany,1995-09-03,195.0,right,Defender - Centre-Back,2017-07-01,2022-06-30,617292.0,212190.0
1988-09-03,26485,Jérôme Boateng,Bayern Munich,Germany,1988-09-03,192.0,right,Defender - Centre-Back,2011-07-14,2021-06-30,6232214.0,183907.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1994-10-01,147540,Ales Cermak,FC Viktoria Plzen,Czech Republic,1994-10-01,181.0,right,Midfielder - Attacking Midfield,2017-07-01,2021-06-30,NaN,244789.0
1992-04-17,384856,Joel Kayamba,FC Viktoria Plzen,DR Congo,1992-04-17,177.0,right,Forward - Right Winger,2018-09-06,2022-06-30,NaN,247045.0
1995-10-17,365861,Ubong Ekpai,FC Viktoria Plzen,Nigeria,1995-10-17,176.0,no_info,Forward - Right Winger,2018-07-01,2021-06-30,NaN,244790.0


In [0]:
players.to_pickle("/content/drive/My Drive/Capstone/Data/Clean/Players_Clean.pkl")